In [45]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import t, ttest_ind
%matplotlib inline

In [46]:
actual = pd.read_csv('gini_1976_2014.csv',skiprows=1,usecols=range(3,37)).to_numpy()[0]

In [47]:
folder = './res/'

In [48]:
filenames = [filename for filename in os.listdir(folder) if filename.endswith('.csv')]

In [49]:
sims = [pd.read_csv(folder+filename)['gini'].to_numpy() for filename in filenames]

In [50]:
biases = [-1.0, -0.75, -0.5, -0.25, 0.0, 0.25, 0.5, 0.75, 1.0]

In [94]:
# Calculating degrees of freedom
def welch_satterthwaite(sample1, sample2):
    n1, n2 = len(sample1), len(sample2)
    s1, s2 = np.std(sample1,ddof=1), np.std(sample2,ddof=1)

    num = ((s1**2)/n1 + (s2**2)/n2)**2
    den = (s1**4)/(n1**3 - n1**2) + (s2**4)/(n2**3 - n2**2)
    return num/den

### Two-tailed t-test (Actual == Sims)

In [95]:
alpha = 0.01

In [96]:
rejects = []

for bias,sim in zip(biases,sims):
    print('\nBias: ', bias)
    print('\nT-test:')
    df = welch_satterthwaite(actual,sim)
    print('\tDegrees of freedom: ',df)
    t_crit = t.ppf(1-(alpha/2),df)
    print('\tT critical: ',t_crit)
    t_test, pval = ttest_ind(actual,sim,equal_var=False)
    print('\tT test: ',t_test)
    print('\tP-value: ',pval)
    rej = (abs(t_test) >= t_crit) and (pval < (alpha/2))
    print('Reject?: ',rej)
    rejects.append(rej)
    print('-'*25)


Bias:  -1.0

T-test:
	Degrees of freedom:  48.4893392742597
	T critical:  2.6810899684288474
	T test:  9.096895503612014
	P-value:  4.6559438919661354e-12
Reject?:  True
-------------------------

Bias:  -0.75

T-test:
	Degrees of freedom:  46.12236381010334
	T critical:  2.6867067890311347
	T test:  6.574879189010737
	P-value:  3.9191468268966075e-08
Reject?:  True
-------------------------

Bias:  -0.5

T-test:
	Degrees of freedom:  46.574827108216816
	T critical:  2.685587238750447
	T test:  3.983248413756641
	P-value:  0.0002370647744803307
Reject?:  True
-------------------------

Bias:  -0.25

T-test:
	Degrees of freedom:  48.91950930559836
	T critical:  2.680129709328738
	T test:  1.097965542724659
	P-value:  0.2775944542685723
Reject?:  False
-------------------------

Bias:  0.0

T-test:
	Degrees of freedom:  52.15093961128722
	T critical:  2.6734403263677917
	T test:  -1.814024689226359
	P-value:  0.07542870856172579
Reject?:  False
-------------------------

Bias:  0.25

T-

### One-tailed test (Sim4 <= Sim3)

In [97]:
df = welch_satterthwaite(sims[4],sims[3])
print('df: ',df)
t_crit = t.ppf(1-alpha,df)
print('T critical: ',t_crit)

df:  196.44549753428475
T critical:  2.345479721443708


In [98]:
t_test, pval = ttest_ind(sims[4],sims[3],equal_var=False)
print('T test: ',t_test)
print('P-value: ',pval)

T test:  4.611817365378075
P-value:  7.1844539679185505e-06


In [99]:
if (t_test >= t_crit) and (pval < alpha):
    print('Reject!')
else:
    print('Failed to reject.')

Reject!
